# Worksheet Solutions


## 2.2c

Complete the following code block to repeat the same procedure for MPI-ESM-LR:


In [ ]:
# Print the current MPI-ESM-LR cube units
print('The current unit for data is: ' + mpiesm.units)

# convert units to kg m-2 day-1
mpiesm.convert_units('kg m-2 day-1')


# Rename the units to mm day-1. Recall that 1 kg m-2 is equivalent to 1 mm of rain
mpiesm.units = 'mm day-1'

# Save the new cube as a new netCDF file using the `outfile` filename we've provided below!
outfile = os.path.join(HISTDIR, 'mpi-esm-lr.mon.1986_2005.GERICS-REMO2015.pr.mmday-1.nc')

iris.save(mpiesm, outfile)

## 2.3e

<b>Answer:</b> Write the line of code required to calculate CHIRPS's (a) standard deviation and (b) annual maximum rainfall in the code block below. <br>
<b>Hint</b>: How could you adapt <code>chirps_ond.aggregated_by(['seasons'], iris.analysis.MEAN)</code> from above? You can refer to the [Iris documentation](https://scitools.org.uk/iris/docs/v2.4.0/iris/iris/analysis.html) if needed.


In [ ]:
# From chirps, calculate: 
# (a) chirps_std 
chirps_std = chirps_ond.aggregated_by(['seasons'], iris.analysis.STD_DEV) 

# (b) chirps_max
chirps_std = chirps.collapsed('time', iris.analysis.MAX) 


# 3.1a


In [ ]:
# Enter the ordered latitude and longitude coordinates for Kuala Lumpur here:
# lon=( , ) # longitude (East - West extent)
# lat=( , ) # latuitude (South - North extent)
lon = (99.5, 100.5)
lat = (13.5, 14.5)

# 3.2e part 1

e) Find the multi-annual monthly means (1986-2005) over the BK area for CHIRPS observations by following the same methodology as above in step d). As before we've given you the input and output files names.


In [ ]:
# Load the BK extracted data created in previous step
infile = os.path.join(CHIRPSDIR, 'chirps.mon.1986_2005.BK.nc')
data = iris.load_cube(infile)

# Add monthly coord categorisation to the time dim coordinate
iris.coord_categorisation.add_month_number(data, 'time', name='month_number')

# Now calculate monthly means
monthly_mean = data.aggregated_by(['month_number'], iris.analysis.MEAN)

# and change units to mm/day
monthly_mean.convert_units('mm day-1')

# create the area averaged monthly mean of daily rainfall
# Find latitude weights
monthly_mean.coord('longitude').guess_bounds()
monthly_mean.coord('latitude').guess_bounds()
grid_areas = iris.analysis.cartography.area_weights(monthly_mean)
# Calculate area averaged monthly mean rainfall
monthly_mean = monthly_mean.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, 
                                      weights=grid_areas)


# Save output
outfile = os.path.join(CLIMDIR, 'chirps.mon.mean.1986_2005.mmday-1.nc')
iris.save(monthly_mean, outfile)

# 3.2e part 2

Plot the CHIRPS observations, compare with the HadGEM2-ES and MPI-ESM-LR driven REMO2015 runs.


In [ ]:
# Load the CHIRPS cube
inpath = os.path.join(CLIMDIR,  'chirps.mon.mean.1986_2005.mmday-1.nc')
cube = iris.load_cube(inpath) 

# Quick line plot for cube 
qplt.plot(cube.coord('month_number'), cube)
plt.title('BK area averaged ' + gcm + '\n monthly average of daily rainfall')
ax = plt.gca()
ax.xaxis.set_label_text('Month Number')
ax.set_xlim(0.5, 12.5)
ax.set_ylim(0, 16)
plt.show()

# 3.3f

Question: Now that the cubes are all on the same grid, what differences do you see? <br> Complete the code template below to help answer this question.


In [ ]:
# Directory name where data is read from
indir = os.path.join(DATADIR, 'EAS-22', 'climatology')

# load HadGEM2-ES downscaled model data
hadgem2 = iris.load_cube(indir + '/hadgem2-es.OND.mean.1986_2005.pr.mmday-1.rg.nc')

# load MPI-ESM-LR downscaled model data
mpiesm = iris.load_cube(indir + '/mpi-esm-lr.OND.mean.1986_2005.pr.mmday-1.rg.nc')

# load CHIRPS data and extract region
chirps = iris.load_cube(indir + '/chirps.OND.mean.1986_2005.pr.mmday-1.nc')
chirps = chirps.intersection(latitude=lat, longitude=lon)

# Do some plotting!
plt.figure(figsize=(12, 10))
plt.subplot(1,3,1)
qplt.pcolormesh(hadgem2[0], vmin=0., vmax=8.)
plt.title('HadGEM2-ES')
plt.gca().coastlines()
plt.subplot(1,3,2)
qplt.pcolormesh(mpiesm[0], vmin=0., vmax=8.)
plt.title('MPI-ESM-LR')
plt.gca().coastlines()
plt.subplot(1,3,3)
qplt.pcolormesh(chirps[0], vmin=0., vmax=8.)
plt.title('CHIRPS')
plt.gca().coastlines()

plt.show()

# 4.2c

Now, repeat the calculations yourself for **temperature**.

First, we calculate the **OND mean** temperatures.


In [ ]:
# HINT: Your filenames should have the format: 
# gcmid + '.OND.mean.' + time_periods[period] + '.GERICS-REMO2015.tm.C.nc'
time_periods = {'historical':'1986_2005', 'future':'2041_2060'}

for gcmid in GCMIDS:
    for period in time_periods.keys():
        # Load the data:
        infile = os.path.join(DATADIR + period + '/' + gcmid + '.mon.' + time_periods[period] + '.GERICS-REMO2015.tm.C.nc')
        data = iris.load_cube(infile)
        
        # In order to calculate OND mean, add a season coordinate, separating OND from the other months:
        iris.coord_categorisation.add_season(data, 'time', name='seasons', seasons=('jfmamjjas','ond'))

        # Extract the data for the OND season only:
        data_ond = data.extract(iris.Constraint(seasons='ond'))

        # Now calculate the mean over the OND season:
        ond_mean = data_ond.collapsed('time', iris.analysis.MEAN)

        # save the OND mean as a netCDF ('outfile' specifies the output file name for your OND mean cube):
        outfile = os.path.join(CLIMDIR, gcmid + '.OND.mean.' + time_periods[period] + '.GERICS-REMO2015.tm.C.nc')
        iris.save(ond_mean, outfile)
        print("saved " + outfile + "\n")
    

# 4.4h

Produce and plot a montly time series of temperature data\*\* relative to the 1986-2005 baseline. As for (f) and (g) produce time series for HadGEM2-ES and MPI-ESM-LR driven runs.


In [ ]:
# Read in the land-sea mask. 
# The cube data array has a land fraction associated with it which we'll use to mask out ocean points.
land_fraction_file = 'sftlf_EAS-22_MOHC-HadGEM2-ES_historical_r0i0p0_GERICS-REMO2015_v1_fx_r0i0p0.nc'
land_fraction = iris.load_cube(DATADIR + land_fraction_file)

# convert this to a binary (i.e. 1 or 0 mask)
land_sea_mask = land_fraction.copy()
land_sea_mask.data = np.where(land_sea_mask.data < 50, 0, 1)
land_sea_mask.name = 'land_binary_mask'
# apply a mask to the cube 
land_sea_mask = iris.util.mask_cube(land_sea_mask, land_sea_mask.data < 0.5)


# Loop over GCMIDS
for gcmid in GCMIDS:
    # Read in original data for baseline and future
    baselinepath = os.path.join(HISTDIR, gcmid + '.mon.1986_2005.GERICS-REMO2015.tm.C.nc')
    futurepath = os.path.join(FUTRDIR, gcmid + '.mon.2041_2060.GERICS-REMO2015.tm.C.nc')
    baseline = iris.load_cube(baselinepath)
    future = iris.load_cube(futurepath)
    
    # Apply land mask
    baseline.data = ma.array(baseline.data, mask=baseline.data*land_sea_mask.data.mask[np.newaxis, :,:])
    future.data = ma.array(future.data, mask=future.data*land_sea_mask.data.mask[np.newaxis, :,:])

    # remove the 2D "true" lat and lon
    baseline.remove_coord('longitude')
    baseline.remove_coord('latitude')
    future.remove_coord('longitude')
    future.remove_coord('latitude')
    
    # Guess bounds
    for cube in [baseline, future]:
        for coord in ['grid_longitude', 'grid_latitude']:
            cube.coord(coord).guess_bounds()
    
    # Calculate mean values over land points
    baseline_land = baseline.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN,
                                      weights = iris.analysis.cartography.area_weights(baseline))
    future_land = future.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN,
                                  weights = iris.analysis.cartography.area_weights(future))

    # Save future & baseline area averaged monthly data (time series)
    baselineout = os.path.join(CLIMDIR, gcmid + '.mon.1986_2005.series.GERICS-REMO2015.tm.C.nc')
    futureout = os.path.join(CLIMDIR, gcmid + '.mon.2041_2060.series.GERICS-REMO2015.tm.C.nc')
    iris.save(baseline_land, baselineout)
    iris.save(future_land, futureout)

    # Subtract baseline from future
    diff = future_land.copy()
    diff.data = future_land.data - baseline_land.data.mean()
    diff.rename('future anomaly relative to mean historical precipitation')

    # Save the area averaged monthly future anomalies (time series)
    outpath = os.path.join(CLIMDIR, gcmid + '.mon.2041_2060.anom.series.GERICS-REMO2015.tm.C.nc')
    iris.save(diff, outpath)
    print('Saved: ' + outpath)    


In [ ]:
# Read in the monthly series
hadgem2es = iris.load_cube(CLIMDIR + '/hadgem2-es.mon.2041_2060.anom.series.GERICS-REMO2015.tm.C.nc')
mpiesm = iris.load_cube(CLIMDIR + '/mpi-esm-lr.mon.2041_2060.anom.series.GERICS-REMO2015.tm.C.nc')
time = hadgem2es.coord('time')

# Plot the two model time series' on the same figure
plt.figure(figsize=(16,5))
iplt.plot(time, hadgem2es, label = 'HadGEM2-ES')
iplt.plot(time, mpiesm, label = 'MPI-ESM-LR')
plt.legend()
plt.suptitle('2041-2060 Temperature anomaly (relative to 1986-2005)')
plt.ylabel('Temperature change ' + hadgem2es.units)
plt.xlabel('Years')
plt.show()

# 5.1b

We'll use CHIRPS as our observational data from which to compare our CORDEX model data.

**Fill in the missing code** to calculate the observed wet days:


In [ ]:
# Load CHIRPS daily precipitation data, but only period of interest
historical_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=1986) 
                            <= cell.point <= PartialDateTime(year=2005))

infile = os.path.join(CHIRPSDIR, 'chirps-v2.0_pr_day_1981-2018_p25.nc')
obs = iris.load_cube(infile, historical_time_constraint)#, 'daily precipitation analysis interpolated onto 0.25deg grids [mm/day]')

# Find number of wet days
chirps_wetdays = obs.collapsed('time', iris.analysis.COUNT, function=lambda values: values > 1)
chirps_wetdays.rename('CHIRPS number of wet days (>=1mm/day)')

# Save wet days cube
outfile = os.path.join(CLIMDIR, 'chirps.wetday.nc')
iris.save(chirps_wetdays, outfile)

# Find number of days in dataset (number_chirps_days)
number_chirps_days = len(obs.coord('time').points)


# Find wet days as percent of all chirps days 
chirps_pcent_wetdays = (chirps_wetdays / number_chirps_days) * 100
chirps_pcent_wetdays.rename('CHIRPS percent of wet days (>=1mm/day)')
chirps_pcent_wetdays.units='%'

# Save 
outfile = os.path.join(CLIMDIR, 'chirps.wetday.pcent.nc')
iris.save(chirps_pcent_wetdays, outfile)


# 5.1e


In [ ]:
    # Add in a line of code to constrain the model domain to these coordinates: 
    # longitude=(70, 160)
    # latitude=(10, 50)
    pcent_change_subset = pcent_change.intersection(longitude=(70, 160), latitude=(10, 50))

    pcent_bias_subset = pcent_bias.intersection(longitude=(70, 160), latitude=(10, 50)) 

# 5.2h

**Plot the differences in the 95th percentiles** between the models and observations, and the future changes in the 95th percentiles of precipitation from both models.

**Complete the code block to plot the figures...**


In [ ]:
# Create a figure of the size 12x12 inches
plt.figure(figsize=(12, 12))

for n, gcmid in enumerate(GCMIDS):
    # HINT: Use the `n` variable to help arrange you plots using: plt.subplot()
    # Use the matplotlib documention to help you! 

    # Load and plot the model bias (model - obs)
    infile = os.path.join(CLIMDIR, gcmid + '.day.pc95.bias.GERICS-REMO2015.pr.mmday-1.nc')
    bias = iris.load_cube(infile)
    plt.subplot(2,2,2*n+1)
    qplt.pcolormesh(bias)
    plt.gca().coastlines()


    # Load and plot the percentage change in precipitation between future and baseline
    infile = os.path.join(CLIMDIR, gcmid + '.day.pc95.diff.GERICS-REMO2015.pr.mmday-1.nc')
    pc_change = iris.load_cube(infile)
    plt.subplot(2,2,2*n+2)
    qplt.pcolormesh(pc_change)
    plt.gca().coastlines()

plt.show()